In [1]:
from transformers import pipeline
from datasets import Dataset
import pandas as pd
import torch
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Load model

In [2]:
pipe = pipeline("question-answering", model="deepset/roberta-base-squad2", device=device)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

load data

### Load data and process it to Dataset#

In [3]:
df = pd.read_csv('/kaggle/input/pmldl-week-12-question-answering/test.csv')

### Predict

In [4]:
predicted_answers = []

for row in tqdm(df.itertuples(), total=len(df), desc="Generating Answers"):
    # Extract the context and question (using itertuples for slightly better performance)
    context = row.context
    question = row.question
    
    result = pipe(question=question, context=context)
    
    predicted_answer = result['answer']
    
    predicted_answers.append(predicted_answer)



Generating Answers: 100%|██████████| 11590/11590 [03:39<00:00, 52.81it/s]


In [5]:
submission_df = pd.DataFrame({
    "id": df["id"],
    "answer": predicted_answers
})

# Display submission
submission_df.head()

,id,answer
0,0,Rutgers University biochemists
1,1,Jonathan Turley
2,2,large quantities of unknown chemicals had been...
3,3,deaths of Frost and Kasza
4,4,January 2006


### Generate submission file

In [6]:
submission_df.to_csv('submission.csv', index=False)